# Introduction

This workbook has two primary goals.
First, it aims to provide sufficient background to understand state-of-the-art papers on cryptographic signatures, with a focus on discrete-logarithm-based multi-party signatures, including their security proofs.
Second, it seeks to develop the skills needed to formalize security notions for cryptographic primitives.
This skill is crucial for selecting appropriate primitives when proposing and reviewing cryptographic protocols, and for defining precisely what a protocol aims to achieve.

The concepts introduced in this section may at first seem abstract, but this level of abstraction is important: it provides the rigorous mathematical framework on which modern provable cryptography is built.
Throughout this workbook we will study algorithms, analyse their running time and success probability, both for explicit algorithms and for hypothetical adversarial algorithms.
In later sections we will sometimes study these algorithms within idealised computational models.

This workbook primarily contains definitions, propositions, and lemmas, with limited intuitive explanations.
Good cryptography papers should include intuition, but it's never complete: intuition is inherently subjective and what makes sense to one reader may not resonate with another.
What you need to do is develop your own explanations while reading the mathematics very precisely.
The goal of this workbook is to build intuition step-by-step through the exercises, which provide hands-on experience with the theoretical concepts.
Some of the most illuminating moments come when we examine how security proofs break after making seemingly minor changes to either the security definition or the cryptographic scheme; these failures reveal the precise assumptions and techniques that make the proofs work.

Note that there is rarely a single standard definition for a concept in cryptography.
Sometimes we encounter equivalent definitions in the literature, while other times we find definitions that differ in subtle but important ways.
This is why contemporary papers in cryptography include a preliminaries section that rigorously defines the concepts they use.

## Prerequisites

This workbook assumes familiarity with basic set notation, operations in $\mathbb{Z}_p$, cyclic groups, elementary probability theory (including conditional probability and the union bound), and proof by contraposition.
See Appendix for a summary of the notation used throughout this book.

## Algorithms

"In mathematics, *algorithm* is commonly understood to be an exact prescription, defining a computational process, leading from various initial data to the desired result." (A. Markov, 1954)

:::{note} Definition: Algorithm
:label: def-algorithm

An algorithm $\mathcal{A}$ is
- *probabilistic* if it gets random bits as input (in addition to its regular input).
  We write $\mathcal{A}(x; \rho)$ to denote running algorithm $\mathcal{A}$ on input $x$ with randomness $\rho$.
  When the randomness is clear from context or unnecessary, we simply write $\mathcal{A}(x)$.
- *polynomial-time* if there exists a polynomial $p$ such that for every $x \in \{0,1\}^\lambda$, $\mathcal{A}(x)$ terminates in time $p(\lambda)$.
- *probabilistic polynomial-time (PPT)* if it is probabilistic and polynomial-time.
:::

Note that the runtime of an algorithm is characterized relative to the size of the input.
All adversaries considered in cryptography are algorithms.

## Negligible Functions

:::{note} Definition: Negligible Function
:label: def-negligible

A function $f: \mathbb{Z} \rightarrow \mathbb{R}$ is *negligible* if for every constant $c > 0$ there exists $N \in \mathbb{N}$ such that for all integers $\lambda > N$, $f(\lambda) < \lambda^{-c}$.
:::

:::{tip} Example
$2^{-\lambda}$ is negligible. $1/\lambda$ is not negligible.
:::

If a function $f$ is negligible in $\lambda$, we write $f(\lambda) = \mathsf{negl}$.

:::{important} Lemma: Properties of negligible functions
:label: lem-negl

1. Let $f$ be a negligible function and $p$ be a polynomial. Then $f(\lambda) \cdot p(\lambda)$ is negligible.
2. Let $f_1, f_2$ be negligible functions. Then $f_1 + f_2$ is negligible.
3. Let $f$ be a negligible function $f(\lambda) \ge 0$ and $k > 0$ be a constant. Then $f + k$ is not negligible.
4. Let $f$ be a non-negligible function and $g$ be a negligible function. Then $f - g$ is not negligible.
:::

The proof of property (1) is left as an exercise (see [](#ex-negl-property-proof)).
The proofs of properties (2)-(4) can be found in the Appendix.

## Games & Asymptotic Security

In this section we will introduce the concept of security games (sometimes called "experiments") and asymptotic security through a series of toy examples.

The $\mathsf{Game}~\mathsf{Guess}$ defined in the figure below takes an algorithm $\mathcal{A}$ and the security parameter $\lambda$ as input and outputs $1$ or $0$.
The game first sets $n = 2^\lambda$ and samples $x$ uniformly at random from $\mathbb{Z}_n$.
Then it runs algorithm $\mathcal{A}$ without input, sets its output to $x'$ and returns $1$ if $x = x'$.

**Guessing game:**

$$
\begin{array}{l}
\underline{\mathsf{Game}~\mathsf{Guess}} \\
n \leftarrow 2^\lambda \\
x \xleftarrow{\$} \mathbb{Z}_n \\
x' \leftarrow \mathcal{A}() \\
\text{return } x = x'
\end{array}
$$

:::{note} Definition: Guessing game advantage
:label: def-guess-adv

For $\mathsf{Game}~\mathsf{Guess}$ as defined above we define the advantage of algorithm $\mathcal{A}$ as
$$
\mathsf{Adv}^{\mathsf{Guess}}_{\mathcal{A}} := \Pr[\mathsf{Game}~\mathsf{Guess} = 1].
$$
:::

:::{important} Proposition
:label: prop-guessing-game

For any algorithm $\mathcal{A}$, $\mathsf{Adv}^{\mathsf{Guess}}_{\mathcal{A}} = \mathsf{negl}$.
:::

:::{dropdown} **Proof**
Let $x'$ be the value returned by $\mathcal{A}$.
The probability that $x$ sampled independently and uniformly at random from $[1, 2^\lambda]$ is equal to $x'$ is $2^{-\lambda} = \mathsf{negl}$.
:::

While the guessing game above provides a clean security definition, formalizing the security of concrete hash functions like SHA256 requires a fundamentally different approach.
To illustrate this, consider the $\mathsf{Game}~\mathsf{SHAPreimgZ}$ defined below, which outputs $1$ if the algorithm $\mathcal{A}$ outputs a preimage of 0 under the hash function $\mathsf{SHA256}: \{0, 1\}^* \rightarrow \{0, 1\}^{256}$.
Note that $\mathcal{A}$ gets $1^{\lambda}$ (a string of 1s of length $\lambda$) as input in order to characterise the running time of $\mathcal{A}$ as a function of the security parameter $\lambda$.

**Game for finding SHA256 preimage of zero:**

$$
\begin{array}{l}
\underline{\mathsf{Game}~\mathsf{SHAPreimgZ}} \\
y \leftarrow 0 \\
x \leftarrow \mathcal{A}(1^\lambda) \\
\text{return } \mathsf{SHA256}(x) = y
\end{array}
$$

:::{important} Proposition
:label: prop-sha-preimg-z

For $\mathsf{Game}~\mathsf{SHAPreimgZ}$ as defined above, let the advantage of $\mathcal{A}$ be defined as
$$
\mathsf{Adv}^{\mathsf{SHAPreIZ}}_{\mathcal{A}} := \Pr[\mathsf{Game}~\mathsf{SHAPreimgZ} = 1].
$$
If there exists $x$ such that $\mathsf{SHA256}(x) = 0$, there exists a PPT algorithm $\mathcal{A}$ such that
$$
\mathsf{Adv}^{\mathsf{SHAPreIZ}}_{\mathcal{A}} = 1.
$$
:::

:::{dropdown} **Proof**
Let $\mathcal{A}$ be an algorithm
$$
\text{return } x
$$
where $\mathsf{SHA256}(x) = 0$.
:::

Next, we consider a variant of preimage resistance for SHA256 where the target value is chosen uniformly at random, preventing the existence of trivial algorithms like the one in [](#prop-sha-preimg-z).
In $\mathsf{Game}~\mathsf{SHAPreimg}$ defined below the value $x$ is a uniformly random bitstring of length $256$.

**Game for finding the SHA256 preimage of a random value:**

$$
\begin{array}{l}
\underline{\mathsf{Game}~\mathsf{SHAPreimg}} \\
x \xleftarrow{\$} \{0, 1\}^{256} \\
y \leftarrow \mathsf{SHA256}(x) \\
x' \leftarrow \mathcal{A}(y) \\
\text{return } \mathsf{SHA256}(x') = y
\end{array}
$$

:::{important} Proposition
:label: prop-sha-preimg

For $\mathsf{Game}~\mathsf{SHAPreimg}$ as defined above, let the advantage of $\mathcal{A}$ be defined as
$$
\mathsf{Adv}^{\mathsf{SHAPreI}}_{\mathcal{A}} := \Pr[\mathsf{Game}~\mathsf{SHAPreimg} = 1].
$$
There exists a PPT algorithm $\mathcal{A}$ such that
$$
\mathsf{Adv}^{\mathsf{SHAPreI}}_{\mathcal{A}} = 1.
$$
:::

:::{dropdown} **Proof**
Let $\mathcal{A}$ be an algorithm that queries $\mathsf{SHA256}$ until it finds a solution as shown below:

**Algorithm for finding the SHA256 preimage of a given value:**

$$
\begin{array}{l}
\underline{\mathcal{A}(y)} \\
x \leftarrow 0 \\
\text{while } \mathsf{SHA256}(x) \neq y \text{ do} \\
\quad x \leftarrow x + 1 \\
\text{return } x
\end{array}
$$

$\mathcal{A}$ will return the correct solution with probability $1$.
The running time of $\mathcal{A}$ is bounded by a constant independent of the security parameter $\lambda$ and, therefore, $\mathcal{A}$ is PPT.
:::

We observe that if the output space of the hash function is independent of the security parameter, then the hash function is not preimage-resistant. This holds even if $\mathsf{Game}~\mathsf{SHAPreimg}$ samples the challenge from $\{0,1\}^\lambda$ instead of the hash function's output space.
For a hash function with constant output size (e.g., 256 bits), an adversary that makes a single random guess has probability at least $2^{-256}$ of finding a preimage, which is non-negligible in $\lambda$.

## Hash Functions

:::{note} Definition: Hash function
:label: def-hash-function

A *hash function* $\mathsf{H}$ is a pair of polynomial-time algorithms $(\mathsf{Gen}, \mathsf{Eval})$ where:
- $\mathsf{Gen}(1^\lambda) \rightarrow \kappa$ is a probabilistic algorithm that takes the security parameter as input and outputs a hashing key $\kappa$. We assume $\kappa$ includes $\lambda$ so that $\mathsf{Eval}$ can run in time polynomial in $\lambda$.
- $\mathsf{Eval}(\kappa, x) \rightarrow y$ is a deterministic algorithm that takes a hashing key $\kappa$ and an input $x \in \{0,1\}^*$ as input and outputs a hash value $y \in S$. When not specified otherwise, we assume $S = \{0,1\}^{\lambda}$. For notational simplicity, $\mathsf{H.Eval}(\kappa, x)$ is often written as $\mathsf{H}(x)$, leaving the hashing key implicit.
:::

Note that the hashing key $\kappa$ is not secret because it is required to evaluate the hash function.

:::{note} Remark: Encoding of Arbitrary Inputs
We extend the notation to arbitrary inputs $x$ (such as group elements, integers, or tuples) and simply write $\mathsf{H.Eval}(\kappa, x)$, implicitly assuming the existence of a fixed injective encoding function that maps the input $x$ into $\{0,1\}^*$.
We omit explicit reference to the encoding function in our notation.
:::

**Game for finding the preimage of a given value under the hash function:**

$$
\begin{array}{l}
\underline{\mathsf{Game}~\mathsf{Preimage}^{\mathsf{H}}} \\
\kappa \leftarrow \mathsf{H.Gen}(1^\lambda) \\
x \xleftarrow{\$} \{0, 1\}^\lambda \\
y \leftarrow \mathsf{H.Eval}(\kappa, x) \\
x' \leftarrow \mathcal{A}(\kappa, y) \\
\text{return } \mathsf{H.Eval}(\kappa, x') = y
\end{array}
$$

Similar to the algorithm described earlier, there exists an algorithm that wins $\mathsf{Game}~\mathsf{Preimage}^{\mathsf{H}}$ with probability 1 by just evaluating $\mathsf{H.Eval}$ on fresh inputs until $\mathsf{H.Eval}(x) = y$.
However, for adequate hash functions this algorithm runs in time exponential in the security parameter.
Hence, the definition of preimage-resistance only considers PPT adversaries.

:::{note} Definition: Preimage-resistance
:label: def-preimage-resistance

A hash function $\mathsf{H}$ is *preimage-resistant* if for any PPT algorithm $\mathcal{A}$,
$$
\mathsf{Adv}^{\mathsf{PreI}}_{\mathcal{A}, \mathsf{H}} := \Pr[\mathsf{Game}~\mathsf{Preimage}^{\mathsf{H}} = 1] = \mathsf{negl}.
$$
:::

:::{note} Remark: Asymptotic Security Framework
Definition [](#def-preimage-resistance) is an example of a definition in the asymptotic security framework.
The hash function $\mathsf{H}$ consists of algorithms that are polynomial-time in $\lambda$, and preimage resistance requires that any PPT adversary's success probability is negligible in $\lambda$.
Hence, by increasing $\lambda$ we can make the success probability of any PPT adversary arbitrarily small.
:::

## Exercises

:::{exercise}
:label: ex-negligible-functions

Determine which of the following functions are negligible in $\lambda$:
1. $f(\lambda) = 0$
2. $f(\lambda) = \frac{1}{2}$
3. $f(\lambda) = \frac{1}{\lambda^2}$
4. $f(\lambda) = \lambda^{4096} \cdot 2^{-\lambda}$
5. $f(\lambda) = \sqrt{2^{-\lambda}}$
:::

:::{exercise}
:label: ex-guessing-game-equivalence

Consider the $\mathsf{Game}~\mathsf{Guess2}$ which is identical to $\mathsf{Game}~\mathsf{Guess}$ except that $x$ is sampled *after* $\mathcal{A}$ is run.
That is, the game first runs $x' \leftarrow \mathcal{A}()$, then samples $x \xleftarrow{\$} \mathbb{Z}_n$, and returns $1$ if $x = x'$.
1. Are these two games equivalent? That is, does $\mathsf{Adv}^{\mathsf{Guess}}_{\mathcal{A}} = \mathsf{Adv}^{\mathsf{Guess2}}_{\mathcal{A}}$ for all algorithms $\mathcal{A}$?
2. Does this change affect the proof of the proposition above?

This exercise illustrates an important principle in game-based proofs: the order of operations does not necessarily affect game outcomes when the operations are independent.
:::

:::{exercise}
:label: ex-sha-preimage-problem

Why is the definition of $\mathsf{Adv}^{\mathsf{SHAPreIZ}}_{\mathcal{A}}$ in [](#prop-sha-preimg-z) problematic from a cryptographic perspective?
What fundamental issue does it illustrate about concrete hash functions like SHA256?
:::

:::{exercise}
:label: ex-sha-random-preimage-problem

Why is the definition of $\mathsf{Adv}^{\mathsf{SHAPreI}}_{\mathcal{A}}$ following [](#prop-sha-preimg-z) also problematic?
Consider the adversary described in the proof.
:::

:::{exercise}
:label: ex-hash-function-definition

How does the definition of preimage resistance for hash functions ([](#def-preimage-resistance)) avoid the issues encountered with the SHA256 examples?
:::

:::{exercise} Optional
:label: ex-negl-property-proof

Prove property (1) of [](#lem-negl): If $f$ is a negligible function and $p$ is a polynomial, then $f(\lambda) \cdot p(\lambda)$ is negligible.

*Hints:*
- Any polynomial $p(\lambda)$ can be bounded by $\lambda^{a}$ for some positive integer $a$ and sufficiently large $\lambda$.
- To show $f(\lambda) \cdot p(\lambda)$ is negligible, for any $c > 0$, use the fact that $f$ is negligible with constant $c + a$.
:::

:::{exercise} Optional
:label: ex-concrete-security

An alternative to the asymptotic approach is the *concrete security* approach.
In this approach, we define $(t,\epsilon)$-hardness as follows:
The $\mathsf{Game}~\mathsf{Guess}$ as defined above is $(t,\epsilon)$-hard if for any algorithm $\mathcal{A}$ running in time $t$, we have $\Pr[\mathsf{Game}~\mathsf{Guess} = 1] \le \epsilon$.

Give a proposition about the concrete hardness of the guessing game, analogous to [](#prop-guessing-game).
:::

:::{exercise} Optional
:label: ex-foundations-hashing

This question explores problems when defining security for hash functions.
Read Section 1 and Section 4 of "Formalizing Human Ignorance" by Rogaway (2006).
1. What is the "Foundations-of-Hashing dilemma"?
2. To get around this, what alternative approach to defining security is suggested?
   What does "explicitly given" mean in this context?
3. (Optional) See also the discussion in Appendix B.7 and the proposed approaches in Appendix B.5 of Bernstein and Lange (2013).
:::